In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
api_key=os.getenv("GOOGLE_MAP_KEY")

In [2]:
import pandas as pd
df_buildings = pd.read_csv("final/final_building_info.csv")

In [ ]:
df_buildings.shape

In [ ]:
df_buildings.head(1)

In [26]:
from googlemaps import Client
import pandas as pd
from collections import defaultdict
import time

def get_nearby_amenities(gmaps_client, location_name, radius_5km=5000, radius_10km=10000):
    """
    Get nearby amenities for a given location using Google Places API
    """
    try:
        geocode_result = gmaps_client.geocode(f"{location_name}, Dubai, UAE")
        
        if not geocode_result:
            return None
            
        location = geocode_result[0]['geometry']['location']
        lat, lng = location['lat'], location['lng']
        
        results = {
            'latitude': lat,
            'longitude': lng,
            'transport': [],
            'metro_stations': [],
            'gyms': [],
            'spas': [],
            'supermarkets': [],
            'malls': [],
            'schools':[],
            'restaurant':[],
            'hospital':[],
            'tourist_attraction':[],
            'park':[],
            'worship':[]
            
        }

        amenity_types = {
            'transport': ['bus_station', 'train_station','taxi_stand'],
            'metro_stations': ['subway_station'],
            'gyms': ['gym'],
            'spas': ['spa','hair_care'],
            'supermarkets': ['supermarket','convenience_store','department_store'],
            'malls': ['shopping_mall'],
            'schools':['school','secondary_school','primary_school','university'],
            'restaurant':['restaurant','cafe'],
            'hospital':['hospital','doctor','drugstore'],
            'tourist_attraction':['tourist_attraction'],
            'park':['park'],
            'worship':['hindu_temple','mosque','church','synagogue']
        }

        
        for amenity_key, place_types in amenity_types.items():
            for place_type in place_types:
                places_result = gmaps_client.places_nearby(
                    location=location,
                    radius=radius_5km,
                    type=place_type
                )
                
                if places_result.get('results'):
                    results[amenity_key] = parse_places_data(places_result,top=5)
        
        
        return results
        
    except Exception as e:
        print(f"Error processing {location_name}: {str(e)}")
        return None

gmaps = Client(key=api_key)

def process_area_buildings(df_building_info, area_name):
    area_buildings = df_building_info[df_building_info['area_name'] == area_name]
    results = []
    
    for _, building in area_buildings.iterrows():
        print(f"Processing {building['building_name']}...")
        
        amenities = get_nearby_amenities(
            gmaps,
            f"{building['building_name']}, Dubai"
        )
        
        if amenities:
            results.append({
                'building_name': building['building_name'],
                'area_name': area_name,
                **amenities
            })
            
        # Sleep to respect API rate limits
        time.sleep(2)
    
    return results

# Process all areas and buildings
all_results = []
for area in df_buildings['area_name'].unique():
    print(f"\nProcessing area: {area}")
    area_results = process_area_buildings(df_buildings, area)
    all_results.extend(area_results)

# Convert results to DataFrame
df_amenities = pd.DataFrame(all_results)

# Save results
df_amenities.to_csv('building_amenities.csv', index=False)

Processing Italy Cluster, International City...
Processing Lady Ratan Manor, International City...
Processing Al Dana 2, International City...
Processing Dragon View, International City...
Processing Riviera Lakeview Apartments, International City...
Processing Al Ayyan Building, International City...
Processing Global Green View 2, International City...

Processing area: Sheikh Zayed Road
Processing Al Attar Tower, Sheikh Zayed Road...
Processing Al Saqr Business Tower, Sheikh Zayed Road...
Processing The Tower, Sheikh Zayed Road...
Processing The Curve Building, Sheikh Zayed Road...
Processing Single Business Tower, Sheikh Zayed Road...
Processing Dubai National Insurance Building, Sheikh Zayed Road...
Processing Conrad Commercial Tower, Sheikh Zayed Road...
Processing Crowne Plaza Commercial Tower, SZR...
Processing Al Moosa Tower 1, Sheikh Zayed Road...
Processing Al Moosa Tower 2, Sheikh Zayed Road...
Processing Dnata Building, Sheikh Zayed Road...
Processing The H Hotel and Offic

In [21]:
def parse_places_data(places_result, top=None):
    parsed_data = []
    
    # Get the results list first
    results = places_result.get('results', [])
    
    # Apply slicing if top is specified
    if top is not None:
        results = results[:top]
    
    for place in results:
        place_data = {
            'business_status': 'UNKNOWN',
            'name': '',
            'rating': 0.0,
            'user_ratings_total': 0,
            'vicinity': '',
            'latitude': None,
            'longitude': None
        }
        
        # Rest of your parsing logic...
        place_data['business_status'] = place.get('business_status', 'UNKNOWN')
        place_data['name'] = place.get('name', '')
        place_data['rating'] = place.get('rating', 0.0)
        place_data['user_ratings_total'] = place.get('user_ratings_total', 0)
        place_data['vicinity'] = place.get('vicinity', '')
        
        geometry = place.get('geometry', {})
        location = geometry.get('location', {})
        place_data['latitude'] = location.get('lat')
        place_data['longitude'] = location.get('lng')
        
        parsed_data.append(place_data)
    
    return parsed_data

In [27]:
df_amenities.head(2)

,building_name,area_name,latitude,longitude,transport,metro_stations,gyms,spas,supermarkets,malls,schools,restaurant,hospital,tourist_attraction,park,worship
0,"Delphine, Dubai Marina",Dubai Marina,25.073213,55.132712,"[{'business_status': 'OPERATIONAL', 'name': 'R...","[{'business_status': 'OPERATIONAL', 'name': 'D...","[{'business_status': 'OPERATIONAL', 'name': 'T...","[{'business_status': 'OPERATIONAL', 'name': 'C...","[{'business_status': 'OPERATIONAL', 'name': 'W...","[{'business_status': 'OPERATIONAL', 'name': 'I...","[{'business_status': 'OPERATIONAL', 'name': 'A...","[{'business_status': 'OPERATIONAL', 'name': 'L...","[{'business_status': 'OPERATIONAL', 'name': 'A...","[{'business_status': 'OPERATIONAL', 'name': 'D...","[{'business_status': 'OPERATIONAL', 'name': 'S...","[{'business_status': 'CLOSED_TEMPORARILY', 'na..."
1,"Marina Plaza, Dubai Marina",Dubai Marina,25.075398,55.139825,"[{'business_status': 'OPERATIONAL', 'name': 'R...","[{'business_status': 'OPERATIONAL', 'name': 'D...","[{'business_status': 'OPERATIONAL', 'name': 'T...","[{'business_status': 'OPERATIONAL', 'name': 'C...","[{'business_status': 'OPERATIONAL', 'name': 'W...","[{'business_status': 'OPERATIONAL', 'name': 'I...","[{'business_status': 'OPERATIONAL', 'name': 'A...","[{'business_status': 'OPERATIONAL', 'name': 'L...","[{'business_status': 'OPERATIONAL', 'name': 'A...","[{'business_status': 'OPERATIONAL', 'name': 'Q...","[{'business_status': 'OPERATIONAL', 'name': 'S...","[{'business_status': 'CLOSED_TEMPORARILY', 'na..."


In [28]:
df_amenities.shape

(247, 16)